# Finding Potential Structural Relatives by Sequence Similarity using proteusPy
Eric G. Suchanek, PhD 2/23/24

Working under the assumption that similar sequence -> similar structure I generated a query on the lowest energy Disulfide Bond in the RCSB database (2q7q) to return PDB IDs for structures with high sequence similarity. I then use some of the proteusPy functions to find structures with similar disulfide bonds.

In [2]:
#
import pandas as pd
import pyvista as pv
from pyvista import set_plot_theme

from proteusPy.Disulfide import Disulfide
from proteusPy.DisulfideList import DisulfideList
from proteusPy.DisulfideLoader import Load_PDB_SS

# pyvista setup for notebooks
pv.set_jupyter_backend("trame")

set_plot_theme("dark")
LIGHT = True

### Load the RCSB Disulfide Database
We load the database and get its properties as follows:

In [3]:
PDB_SS = Load_PDB_SS(verbose=True)
PDB_SS.describe()

-> load_PDB_SS(): Reading /Users/egs/miniforge3/envs/proteusPy/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
-> load_PDB_SS(): Done reading /Users/egs/miniforge3/envs/proteusPy/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
    =========== RCSB Disulfide Database Summary ==============
       =========== Built: 2024-03-16 17:12:24 ==============
PDB IDs present:                    35818
Disulfides loaded:                  120494
Average structure resolution:       2.34 Å
Lowest Energy Disulfide:            2q7q_75D_140D
Highest Energy Disulfide:           1toz_456A_467A
Cα distance cutoff:                 8.00 Å
Total RAM Used:                     30.72 GB.
    ================= proteusPy: 0.93.0 =======================


In [7]:
ss = PDB_SS['5rsa_26A_84A']
print(f'{ss.name}')
ss.dihedrals

5rsa_26A_84A


[-68.64177700691641,
 -87.08310517280916,
 -81.44489804423738,
 -50.83886936309293,
 -66.09666929641922]

In [ ]:
best_ss = PDB_SS["2q7q_75D_140D"]
best_ss.pprint()
best_ss.display(style="sb", light=LIGHT)

I generated a query on: https://www.ebi.ac.uk/pdbe/entry/pdb/2q7q to return PDB IDs for structures with high sequence similarity to 2q7q - the protein with the lowest energy disulfide bond in the RCSB database. This yielded a ```.csv``` file, which we will import below:

In [ ]:
ss_df = pd.read_csv("2q7q_seqsim.csv")
ss_df.head(5)

All of the nearest sequence neighbors are sadly, bacterial. Let's extract the unique ids next.

In [ ]:
relative_list = ss_df["pdb_id"].unique()
relative_list

We now need to convert the list of PDB IDs into real disulfides from the database. We do this with the ``DisulfideLoader.build_ss_from_idlist()`` function. Next we print out some relevant statistics.


In [ ]:
relatives = DisulfideList([], "relatives")
relatives = PDB_SS.build_ss_from_idlist(relative_list)

print(
    f"There are: {relatives.length} related structures.\nAverage Energy: {relatives.Average_Energy:.2f} kcal/mol\nAverage Ca distance: {relatives.Average_Distance:.2f} Å"
)
print(
    f"Average resolution: {relatives.Average_Resolution:.2f} Å \nAverage torsion distance: {relatives.Average_Torsion_Distance:.2f}°"
)

Now let's look at the lowest and highest energy structures in this list of relatives.

In [ ]:
ssmin, ssmax = relatives.minmax_energy
duolist = DisulfideList([ssmin, ssmax], "mM")
# duolist.display(style='sb', light=LIGHT)

In [ ]:
duolist.display_overlay(light=LIGHT)

The two Disulfides...

We can find disulfides that are conformationally related by using the DisulfideList.nearest_neighbors() function with a dihedral angle cutoff. This cutoff is measure of angular similarity across all five sidechain dihedral angles.

In [ ]:
close_neighbors = relatives.nearest_neighbors(
    ssmin.chi1, ssmin.chi2, ssmin.chi3, ssmin.chi4, ssmin.chi5, 10.0
)
close_neighbors.length

In [ ]:
# close_neighbors.display_overlay(light=LIGHT)

So now we have the 18 close neighbors of the lowest energy structure.

In [ ]:
ssTotList = PDB_SS.SSList
global_neighbors = ssTotList.nearest_neighbors(
    ssmin.chi1, ssmin.chi2, ssmin.chi3, ssmin.chi4, ssmin.chi5, 5.0
)
global_neighbors.length

In [ ]:
global_neighbors.display_overlay(light=LIGHT)

In [ ]:
from scipy.optimize import minimize
import numpy as np


def ss_energy_function(x):
    chi1, chi2, chi3, chi4, chi5 = x
    energy = 2.0 * (
        np.cos(np.deg2rad(3.0 * chi1)) + np.cos(np.deg2rad(3.0 * chi5))
    )
    energy += np.cos(np.deg2rad(3.0 * chi2)) + np.cos(np.deg2rad(3.0 * chi4))
    energy += (
        3.5 * np.cos(np.deg2rad(2.0 * chi3))
        + 0.6 * np.cos(np.deg2rad(3.0 * chi3))
        + 10.1
    )
    return energy


initial_guess = [
    -60.0,
    -60.0,
    -90.0,
    -60.0,
    -60.0,
]  # initial guess for chi1, chi2, chi3, chi4, chi5
result = minimize(ss_energy_function, initial_guess, method="Nelder-Mead")
minimum_energy = result.fun
inputs = result.x

print(
    f'Minimum Energy: {minimum_energy:.3f} for conformation: {[f"{x:.3f}" for x in inputs]}'
)